## ORCIDS/DOIs with Python

In [97]:
#additional modules to add to the env. file

#!pip install filetype
#!pip install datacite
#!pip install habanero
#!pip install insights-core
#! pip install -U py-cpuinfo
#!pip install git+https://github.com/vdmitriyev/pyorcid.git
#!pip install xmltodict

In [98]:
from habanero import Crossref
import datacite
from datacite import DataCiteRESTClient, schema42
from insights.parsers.cpuinfo import CpuInfo
from insights.parsers import cpuinfo
from insights.tests import context_wrap
from habanero import *


In [99]:
import json

In [116]:
from secret import ORCID_ID, ORCID_PW

# ORCIDS


In [100]:
from requests import RequestException
import orcid #official library

In [117]:
api = orcid.PublicAPI(ORCID_ID, ORCID_PW,
                      sandbox=False)



#You will then be returned an access token similar to the following. 
#The token returned is long-lived (not expiring for approximately 20 years) and can be used multiple times to 
#retrieve public data from ORCID records. 

new_token = False
if new_token:
    search_token = api.get_search_token_from_orcid()
else:
    search_token = '66e55f4c-b62a-4eef-9c7c-2a8dee553ccc'

#how long does a search token last
#here is a previous one...
#'66e55f4c-b62a-4eef-9c7c-2a8dee553ccc'

search_token

'66e55f4c-b62a-4eef-9c7c-2a8dee553ccc'

In [118]:
#Each item (work, funding, employment etc) has a put code. 
#This can be used to obtain full details of the item if needed.

In [119]:
#Test Sara's ORCID 
summary = api.read_record_public('0000-0002-1270-4377', 'record',
                                 search_token)


In [120]:
type(summary )

dict

In [121]:
summary.keys()

dict_keys(['orcid-identifier', 'preferences', 'history', 'person', 'activities-summary', 'path'])

In [122]:
for key in summary['activities-summary'].keys():
    try:
        print(key, summary['activities-summary'][key].keys())
    except:
        print(key,summary['activities-summary'][key])

last-modified-date dict_keys(['value'])
educations dict_keys(['last-modified-date', 'education-summary', 'path'])
employments dict_keys(['last-modified-date', 'employment-summary', 'path'])
fundings dict_keys(['last-modified-date', 'group', 'path'])
peer-reviews dict_keys(['last-modified-date', 'group', 'path'])
works dict_keys(['last-modified-date', 'group', 'path'])
path /0000-0002-1270-4377/activities


In [30]:
len(summary['activities-summary']['works'])

3

In [23]:
json_string = json.dumps(summary)

In [40]:
#Test Sara's ORCID 
#summary_works = api.read_record_public('0000-0002-1270-4377', 'record',
#                                 search_token)

work = api.read_record_public('0000-0002-1270-4377', 'works', search_token)

In [88]:
#work

In [6]:
# a different way

# DOIs

## Habanero (crossref) - articles

Seems to works well for articles. 

In [54]:
cr = Crossref()
output =  cr.works(ids = 'https://doi.org/10.1029/2022GC010386')

In [55]:
type(output)

dict

In [56]:
output.keys()

dict_keys(['status', 'message-type', 'message-version', 'message'])

In [57]:
output['status'], output['message-type'], 

('ok', 'work')

In [59]:
#the record is contained in the message value,.  
#it is a dict
#type(output['message'])
print(output['message'].keys() )

dict_keys(['indexed', 'reference-count', 'publisher', 'issue', 'license', 'funder', 'content-domain', 'short-container-title', 'published-print', 'abstract', 'DOI', 'type', 'created', 'update-policy', 'source', 'is-referenced-by-count', 'title', 'prefix', 'volume', 'author', 'member', 'published-online', 'reference', 'container-title', 'original-title', 'language', 'link', 'deposited', 'score', 'resource', 'subtitle', 'short-title', 'issued', 'references-count', 'journal-issue', 'alternative-id', 'URL', 'archive', 'relation', 'ISSN', 'issn-type', 'subject', 'published', 'assertion'])


In [61]:
#Get the first Author
output['message']['author'][0]

{'ORCID': 'http://orcid.org/0000-0002-1521-7910',
 'authenticated-orcid': False,
 'given': 'Andrés Felipe',
 'family': 'Rodríguez Corcho',
 'sequence': 'first',
 'affiliation': [{'name': 'School of Geography, Earth and Atmospheric Sciences University of Melbourne  Melbourne VIC Australia'},
  {'name': 'School of Geosciences University of Sydney  Sydney NSW Australia'}]}

In [62]:
len(output['message']['author'])

6

In [63]:
#habanero cannot read orcidIDs
#output = cr.works(ids = 'http://orcid.org/0000-0002-1270-4377')

## Habanero (crossref) - other DOIs

* use the content_negotiation function, you get something
* Supports DOIs from Crossref, Datacite and Medra


### GeoNetwork

In [102]:
# Geonetwork
#this is the DOI for Gadi from the GeoNetwork record. 

output =  cn.content_negotiation('https://doi.org/10.25914/608bfd1838db2', 
                                format = "datacite-xml")

In [103]:
import xmltodict
xmltodict.parse(output)

{'resource': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xmlns': 'http://datacite.org/schema/kernel-4',
  '@xsi:schemaLocation': 'http://datacite.org/schema/kernel-4 http://schema.datacite.org/meta/kernel-4/metadata.xsd',
  'identifier': {'@identifierType': 'DOI', '#text': '10.25914/608BFD1838DB2'},
  'creators': {'creator': {'creatorName': {'@nameType': 'Organizational',
     '#text': 'National Computational Infrastructure'},
    'nameIdentifier': {'@nameIdentifierScheme': 'ROR',
     '@schemeURI': 'https://ror.org',
     '#text': 'https://ror.org/04yx6dh41'}}},
  'titles': {'title': 'Gadi Supercomputer'},
  'publisher': 'NCI Australia',
  'publicationYear': '2021',
  'resourceType': {'@resourceTypeGeneral': 'Service', '#text': 'Service'},
  'dates': {'date': {'@dateType': 'Issued', '#text': '2021'}},
  'language': 'en',
  'sizes': None,
  'formats': None,
  'version': None}}

In [104]:
#cn.content_negotiation?

### Zenodo

In [105]:
#cn.content_negotiation?

output =  cn.content_negotiation('https://doi.org/10.5281/zenodo.5772203',
                                format = "datacite-xml")


In [106]:
xmltodict.parse(output)['resource'].keys()

dict_keys(['@xmlns:xsi', '@xmlns', '@xsi:schemaLocation', 'identifier', 'creators', 'titles', 'publisher', 'publicationYear', 'resourceType', 'dates', 'alternateIdentifiers', 'relatedIdentifiers', 'sizes', 'formats', 'version', 'rightsList', 'descriptions'])

In [108]:
xmltodict.parse(output)['resource']

{'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
 '@xmlns': 'http://datacite.org/schema/kernel-4',
 '@xsi:schemaLocation': 'http://datacite.org/schema/kernel-4 http://schema.datacite.org/meta/kernel-4/metadata.xsd',
 'identifier': {'@identifierType': 'DOI', '#text': '10.5281/ZENODO.5772203'},
 'creators': {'creator': {'creatorName': {'@nameType': 'Personal',
    '#text': 'Corcho, Andres Felipe Rodriguez'},
   'givenName': 'Andres Felipe Rodriguez',
   'familyName': 'Corcho',
   'affiliation': 'University of Melbourne'}},
 'titles': {'title': 'andresrcorcho/Dynamics-of-Arc-Continent-Collision: Dynamics of arc-continent collision'},
 'publisher': 'Zenodo',
 'publicationYear': '2021',
 'resourceType': {'@resourceTypeGeneral': 'Software',
  '#text': 'SoftwareSourceCode'},
 'dates': {'date': {'@dateType': 'Issued', '#text': '2021-12-10'}},
 'alternateIdentifiers': {'alternateIdentifier': {'@alternateIdentifierType': 'URL',
   '#text': 'https://zenodo.org/record/5772203'}},
 'rela

### Figshare

In [111]:
output =  cn.content_negotiation('https://doi.org/10.6084/m9.figshare.21506217',
                                format = "datacite-xml")





In [112]:
xmltodict.parse(output)['resource']

{'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
 '@xmlns': 'http://datacite.org/schema/kernel-4',
 '@xsi:schemaLocation': 'http://datacite.org/schema/kernel-4 http://schema.datacite.org/meta/kernel-4/metadata.xsd',
 'identifier': {'@identifierType': 'DOI',
  '#text': '10.6084/M9.FIGSHARE.21506217'},
 'creators': {'creator': [{'creatorName': 'Chen, Jie',
    'givenName': 'Jie',
    'familyName': 'Chen',
    'nameIdentifier': {'@nameIdentifierScheme': 'ORCID',
     '@schemeURI': 'https://orcid.org',
     '#text': 'https://orcid.org/0000-0002-4674-8351'}},
   {'creatorName': 'Olive, Jean-Arthur',
    'givenName': 'Jean-Arthur',
    'familyName': 'Olive',
    'nameIdentifier': {'@nameIdentifierScheme': 'ORCID',
     '@schemeURI': 'https://orcid.org',
     '#text': 'https://orcid.org/0000-0001-6899-592X'}},
   {'creatorName': 'Cannat, Mathilde',
    'givenName': 'Mathilde',
    'familyName': 'Cannat'}]},
 'titles': {'title': "Research Data for 'Beyond spreading rate: controls on

In [113]:
#### Pangaea

In [114]:

output =  cn.content_negotiation('https://doi.org/10.1594/PANGAEA.900982',
                                format = "datacite-xml")



In [115]:
xmltodict.parse(output)['resource']

{'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
 '@xmlns': 'http://datacite.org/schema/kernel-4',
 '@xsi:schemaLocation': 'http://datacite.org/schema/kernel-4 http://schema.datacite.org/meta/kernel-4/metadata.xsd',
 'identifier': {'@identifierType': 'DOI', '#text': '10.1594/PANGAEA.900982'},
 'creators': {'creator': [{'creatorName': 'Galerne, Christophe',
    'givenName': 'Christophe',
    'familyName': 'Galerne',
    'nameIdentifier': {'@nameIdentifierScheme': 'ORCID',
     '@schemeURI': 'https://orcid.org',
     '#text': 'https://orcid.org/0000-0001-9696-205X'}},
   {'creatorName': 'Hasenclever, Jörg',
    'givenName': 'Jörg',
    'familyName': 'Hasenclever'}]},
 'titles': {'title': 'Hydrothermal fluid flow modelling in homogeneous and stratified sedimentary basin'},
 'publisher': 'PANGAEA',
 'publicationYear': '2019',
 'resourceType': {'@resourceTypeGeneral': 'Dataset',
  '#text': 'Supplementary Dataset'},
 'subjects': {'subject': [{'@subjectScheme': 'Parameter',
    '#t

# CPU Info

Exploring how to generate system info in a consistemn manner. 

workhorsy/py-cpuinfo

* looks good - it works across a range of sytems, etc
* but its opertaion is limited to work on the host computer, not on files. Would need to be tweaked. 
* `./ASSETS/cpu_info.json` is the result of runnign this script Gadi


In [109]:
from cpuinfo import *

In [110]:
cpu_here = get_cpu_info()
cpu_here

{'python_version': '3.10.13.final.0 (64 bit)',
 'cpuinfo_version': [9, 0, 0],
 'cpuinfo_version_string': '9.0.0',
 'arch': 'X86_64',
 'bits': 64,
 'count': 8,
 'arch_string_raw': 'x86_64',
 'vendor_id_raw': 'GenuineIntel',
 'brand_raw': 'Intel(R) Xeon(R) CPU E3-1505M v6 @ 3.00GHz',
 'hz_advertised_friendly': '3.0000 GHz',
 'hz_actual_friendly': '3.6000 GHz',
 'hz_advertised': [3000000000, 0],
 'hz_actual': [3600024000, 0],
 'stepping': 9,
 'model': 158,
 'family': 6,
 'flags': ['3dnowprefetch',
  'abm',
  'acpi',
  'adx',
  'aes',
  'aperfmperf',
  'apic',
  'arat',
  'arch_capabilities',
  'arch_perfmon',
  'art',
  'avx',
  'avx2',
  'bmi1',
  'bmi2',
  'bts',
  'clflush',
  'clflushopt',
  'cmov',
  'constant_tsc',
  'cpuid',
  'cpuid_fault',
  'cx16',
  'cx8',
  'de',
  'ds_cpl',
  'dtes64',
  'dtherm',
  'dts',
  'epb',
  'ept',
  'ept_ad',
  'erms',
  'est',
  'f16c',
  'flexpriority',
  'flush_l1d',
  'fma',
  'fpu',
  'fsgsbase',
  'fxsr',
  'hle',
  'ht',
  'hwp',
  'hwp_act_w

In [96]:
cpu_here.keys()

dict_keys(['python_version', 'cpuinfo_version', 'cpuinfo_version_string', 'arch', 'bits', 'count', 'arch_string_raw', 'vendor_id_raw', 'brand_raw', 'hz_advertised_friendly', 'hz_actual_friendly', 'hz_advertised', 'hz_actual', 'stepping', 'model', 'family', 'flags', 'l3_cache_size', 'l2_cache_size', 'l1_data_cache_size', 'l1_instruction_cache_size', 'l2_cache_line_size', 'l2_cache_associativity'])

## Scratch

In [ ]:
import pyorcid #https://github.com/vdmitriyev/pyorcid
#retrieve john's profile from his ORCID


author= pyorcid.get('0000-0002-1270-4377')
print (author.family_name)

author.biography
author.publications[0]
author = pyorcid.get('0000-0003-3566-1557')
pubs = author.publications
pub1 = pubs[0]
pub1.citation_value

In [ ]:

RedHatInsights/insights-core

* has the desired operation, in that it reads in a CPUinfo file. 
* But it doesn't give the right info for my system,


In [ ]:
#with open('./ASSETS/cpuinfo_gadi', 'r') as file:
with open('./ASSETS/cpuinfo_local', 'r') as file:    
    data = file.read().replace('\n', '')
    cpu_info = CpuInfo(context_wrap(data))